# Code for scrapping data

In [1]:
import AO3
import pandas as pd
import numpy as np
import time

In [2]:
#Testing code snippet to grab titles and summaries to save in dataframe
url = "https://archiveofourown.org/works/11233080"
workid = AO3.utils.workid_from_url(url)
work = AO3.Work(workid)
title = work.title
summary = work.summary

In [2]:
titles = []
summaries = []

for n in range(18000, 18201):
    time.sleep(15)
    search = AO3.Search(single_chapter = True, word_count=AO3.utils.Constraint(2500, 30000), language = 'en', page = n)
    search.update()
    for result in search.results:
        titles.append(result.title)
        try:
            summaries.append(result.summary)
        except:
            summaries.append('No summary')

In [3]:
df = pd.DataFrame(zip(titles, summaries), columns = ['title', 'summary'])

In [4]:
df.to_csv('../data/frame_008', index = False)

# Code for cleaning

## Check for duplicate rows

In [65]:
no_dupes = df[~df.duplicated()]

In [55]:
import re
from nltk.tokenize import word_tokenize
import string

punct = list(string.punctuation)

def clean_summaries(text):
    #Get rid of \n at start and end of paragraph
    text = re.sub(r'^\\n|\\n$', '', text)
    #Capture groupings like \xa0
    text = re.sub(r'\\', r' ', text)
    text = re.sub(r'\\(\w+)', '', text)
    #Normalize whitespace
    text = re.sub("\s\s+", " ", text)
    text_token = word_tokenize(text)
    return [w.lower() for w in text_token if w not in punct]

In [30]:
#Check if english title/summary
from langdetect import detect

answer = detect("Sentence to detect")
if answer != 'en':
    #assign 1
else:
    #assign 0
    
#if summary == 'No summary':
    #add flag to ignore row and/or drop

IndentationError: expected an indented block (3121125942.py, line 7)

# Code to train NN to generate new titles/summaries

Merge all summaries + titles together to one long string
Base train length on average length of summary/title

In [ ]:
train_len = 4 + 1

text_sequences = []
for i in range(train_len, len(tokens)):
    seq = tokens[i - trian_len:i]
    text_sequences.append(seq)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenzier.fit_on_texts(text_sequences)

vocabulary_size = len(tokenizer.word_counts())

In [ ]:
sequences = tokenizer.texts_to_sequences(text_sequences)
sequences = np.array(sequences)

Create X and y dataframes

In [ ]:
from keras.utils import to_categorical

X = sequences[:, :-1]
y = sequences[:, -1]

y = to_categorical(y, num_classes = vocabulary_size + 1)
seq_len = X.shape[1]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

neurons = train_len * 3
dense_neurons = train_len * 4

def create_model(vocabulary_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocabulary_size, seq_len, input_length = seq_len))
    model.add(LSTM(neurons, return_sequences = True))
    model.add(LSTM(neurons))
    model.add(Dense(dense_neurons, activation = 'relu'))
    
    model.add(Dense(vocabulary_size, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    model.summary()
    
    return model

In [ ]:
model = create_model(vocabulary_size + 1, seq_len)

In [ ]:
from pickle import dump, load

model.fit(X, y, batch_size = 128, epochs = 50, verbose = 1)

In [ ]:
model.save('fic_title_generator.h5')

In [ ]:
dump(tokenizer, open('title_tokens', 'wb'))

## Generate new text

In [ ]:
from keras.preprocessing.sequences import pad_sequences

#Make starter list for seed_text and choose random starter point

def generate_title(model, tokenizer, seq_len, seed_text, num_gen_words):
    
    output_text = []
    input_text = seed_text
    
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen = seq_len, truncating = 'pre')
        pred_word_idx = model.predict_classes(pad_encoded, verbose = 0)[0]
        pred_word = tokenzier.index_word[pred_word_idx]
        
        input_text += ' ' + pred_word
        output_text.append(pred_word)
    
    return " ".join(output_text)

In [ ]:
from keras.models import load_model

model = load_model('fic_title_generator.h5')
tokenizer = load(open('title_tokens', 'rb'))